1. Load the model Yeast8

In [2]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite

model = read_sbml_model('yeastGEM.xml')

2. Running model with Standard Medium

In [3]:
#Create reaction dictionary

In [4]:
xyodict = {1: "r_2104", # xylitol transport
           2: "r_1719", # D-xylose transport
           3: "r_1093", # xylose reductase
           4: "r_1092", # xylitol dehydrogenase
           5: "r_0889", # 6-phosphogluconate dehydrogenase
           6: "r_0466", # Glucose-6-phosphate dehydrogenase
          }

In [5]:
#Setting different medium conditions. The elements in the different mediums can be displayed replacing the pertinent number in the bottom code

In [6]:
medium_glu = model.medium  # 100% glucose as carbon source in the medium

medium_glu50 = model.medium # 50% glucose and 50% xylose as carbon sources in the medium
with model: 
    medium_glu50['r_1714'] = 0.5
    medium_glu50['r_1718'] = 0.5

for element in medium_glu.keys():
    print(model.reactions.get_by_id(element).name, model.reactions.get_by_id(element), medium_glu[element])

ammonium exchange r_1654: s_0420[e] <=>  1000.0
D-glucose exchange r_1714: s_0565[e] <=>  1.0
H+ exchange r_1832: s_0796[e] <=>  1000.0
iron(2+) exchange r_1861: s_0925[e] <=>  1000.0
oxygen exchange r_1992: s_1277[e] <=>  1000.0
phosphate exchange r_2005: s_1324[e] <=>  1000.0
potassium exchange r_2020: s_1374[e] <=>  1000.0
sodium exchange r_2049: s_1438[e] <=>  1000.0
sulphate exchange r_2060: s_1468[e] <=>  1000.0
water exchange r_2100: s_0805[e] <=>  1000.0
chloride exchange r_4593: s_4200[e] <=>  1000.0
Cu2(+) exchange r_4594: s_4201[e] <=>  1000.0
Mn(2+) exchange r_4595: s_4202[e] <=>  1000.0
Zn(2+) exchange r_4596: s_4203[e] <=>  1000.0
Mg(2+) exchange r_4597: s_4204[e] <=>  1000.0
Ca(2+) exchange r_4600: s_4199[e] <=>  1000.0


3. Run the model with the selected medium. In this model there are already included a combination of the different mutations that have been proven to work individually with promise to improve xylitol production. This include upregulation of a xylose transporter form the medium to inside of the cell, an upregulation of the xylose reductase, for a higher xylitol conversion, a knock out of the gene XDH in the PPP pathway to prevent xylitol to keep going down the pathway, and an upregulation of the xylitol transporter. In the code below, we can combine these modifications with mediums with different composition, using alternative proportions of the two selected carbon sources: glucose and xylose. The idea behind this experiment is to determine the best conditions for xylitol production while still having cell growth.

In [9]:
with model:
    
    #MODEL MEDIUM
    model.medium = medium_glu50 # 60% glucose and 40% xylose as carbon sources in the medium

    #BUILD THE MUTANTS
    model.reactions.r_1719.bounds = (0,10) # regulation for xylose transporter
    model.reactions.r_1093.bounds = (0,10) # regulation for xylose reductase
    model.genes.YLR070C.knock_out() # knockout for xylitol dehydrogenase
    model.reactions.r_2104.bounds = (0.75*medium_glu50['r_1718'],1) # regulation for xylitol transporter
    model.reactions.r_0889.bounds = (0,10) # 6-phosphogluconate dehydrogenase
    model.reactions.r_0466.bounds = (0,10) # Glucose-6-phosphate dehydrogenase

    
    ## solve model
    # model.objective = model.reactions.r_2104 # objective is maximum xylitol production
    solution3 = model.optimize()
    print(model.summary())
    for key in xyodict.keys():
        print(model.reactions.get_by_id(xyodict[key]).name, model.reactions.get_by_id(xyodict[key]).flux)
    print(model.reactions.r_2111.name, model.reactions.r_2111.flux)
    print(model.reactions.r_1714.name, model.reactions.r_1714.flux)
    print(model.reactions.r_1166.name, model.reactions.r_1166.flux)

Objective
1.0 r_2111 = 0.037706719837873166

Uptake
------
Metabolite Reaction      Flux  C-Number  C-Flux
 s_0420[e]   r_1654    0.2696         0   0.00%
 s_0565[e]   r_1714       0.5         6  61.54%
 s_0579[e]   r_1718     0.375         5  38.46%
 s_0796[e]   r_1832   0.01907         0   0.00%
 s_0925[e]   r_1861 1.146E-06         0   0.00%
 s_1277[e]   r_1992     1.124         0   0.00%
 s_1324[e]   r_2005    0.2454         0   0.00%
 s_1374[e]   r_2020 0.0001369         0   0.00%
 s_1438[e]   r_2049 0.0001497         0   0.00%
 s_1468[e]   r_2060  0.003243         0   0.00%
 s_4200[e]   r_4593 4.864E-05         0   0.00%
 s_4201[e]   r_4594 2.485E-05         0   0.00%
 s_4202[e]   r_4595 0.0001029         0   0.00%
 s_4203[e]   r_4596  2.82E-05         0   0.00%
 s_4204[e]   r_4597 4.685E-05         0   0.00%
 s_4199[e]   r_4600 8.182E-06         0   0.00%

Secretion
---------
Metabolite Reaction       Flux  C-Number  C-Flux
 s_0458[e]   r_1672     -1.375         1  42.30%
 s_077